# Capacity Factor
---

In [ ]:
import sys
import warnings

sys.path.append("../../")
warnings.simplefilter("ignore")

In [ ]:
import analyze
from plot.analyze_pg import AnalyzePG
from westernintnet.westernintnet import win_data
import pandas as pd
import matplotlib.pyplot as plt
plt.ioff()

In [ ]:
def get_plant_id(zone, resource):
    plant_id = []
    if zone == 'Western':
        try:
            plant_id = win_data.genbus.groupby('type').get_group(resource).index.values.tolist()
        except KeyError:
            pass
    elif zone == 'California':
        CA = ['Bay Area', 'Central California', 'Northern California',
              'Southeast California', 'Southwest California']
        for load_zone in CA:
            try:
                plant_id += win_data.genbus.groupby(['ZoneName', 'type']).get_group(
                    (load_zone, resource)).index.values.tolist()
            except KeyError:
                pass
    else:
        try:
            plant_id = win_data.genbus.groupby(['ZoneName', 'type']).get_group(
                (zone, resource)).index.values.tolist()
        except KeyError:
            pass
    
    return plant_id

In [ ]:
def prepare_data(scenario):
    #renewable profiles
    win_data.read_solar_data('sam')
    win_data.read_wind_data()
    
    if scenario is "2016":
        # generation
        PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG.pkl')
        # demand
        win_data.read_demand_data('v3')
        # renewable multipliers
        multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
    elif scenario is "ca2020" or scenario is "ca2020_congestion":
        # generation
        if scenario is "ca2020":
            PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2020.pkl')
        else:
            PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2020_CALineCapacityX2.pkl')
        # demand
        win_data.read_demand_data('cal2020')
        # renewable multipliers
        factor = [1.52, 1.48]
        multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
        for i, resource in enumerate(['solar', 'wind']):
            plant_id = get_plant_id('California', resource)
            multiplier.loc[plant_id] = factor[i]
    elif scenario is "ca2030" or scenario is "ca2030_congestion":
        # generation
        if scenario is "ca2030":
            PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030.pkl')
        else:
            PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030_CALineCapacityX2.pkl')
        # demand
        win_data.read_demand_data('cal2030')
        # renewable multipliers
        factor = [2.3, 2.3]
        multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
        for i, resource in enumerate(['solar', 'wind']):
            plant_id = get_plant_id('Western', resource)
            multiplier.loc[plant_id] = factor[i]
    elif scenario is "west2020":
        # generation
        PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2020_WestTarget.pkl')
        # demand
        win_data.read_demand_data('cal2020')
        # renewable multipliers
        factor = [2.25, 2.25]
        multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
        for i, resource in enumerate(['solar', 'wind']):
            plant_id = get_plant_id('Western', resource)
            multiplier.loc[plant_id] = factor[i]
    elif scenario is "west2030":
        # generation
        PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030_WestTarget.pkl')
        # demand
        win_data.read_demand_data('cal2030')
        # renewable multipliers
        factor = [5, 5]
        multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
        for i, resource in enumerate(['solar', 'wind']):
            plant_id = get_plant_id('Western', resource)
            multiplier.loc[plant_id] = factor[i]
    
    return (PG, win_data, multiplier)